In [1]:
import csv
import re

In [2]:
from collections import defaultdict

In [3]:
gene_fe_no = defaultdict(lambda: 0)
gene_fe_ss_no = defaultdict(list)

In [4]:
with open("../data/ref/homo_sapiens/annot/out/meta_exon.csv", 'r') as f:
    reader = csv.reader(f)
    next(reader)
    
    for row in reader:
        gene, gene_func, chrom, beg, end, strand, meta_type, make_up = row

        if "FE" not in meta_type:
            continue
        
        ss_set = set()
        
        for exon in make_up.split(","):
            exon_type, beg, end = re.split('[=:-]', exon)
            
            if exon_type == "FE":
                if strand == "+":
                    ss_set.add(end)
                else:
                    ss_set.add(beg)

        strand = int(strand == "+")
        meta_exon = f"{chrom}:{beg}-{end}:{strand}"

        gene_fe_no[gene] += 1
        gene_fe_ss_no[gene].append((meta_exon, len(ss_set)))

In [5]:
with open("../data/ref/homo_sapiens/annot/out/fe_stat.csv", 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['gene', 'fe_no', 'ss_no', 'make_up'])
    
    for gene, fe_no in gene_fe_no.items():        
        writer.writerow(
            [gene, fe_no, sum([n for _, n in gene_fe_ss_no[gene]])] + [";".join([f"{x}={n}" for x, n in gene_fe_ss_no[gene]])]
        )